In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0,org.apache.spark:spark-sql-kafka-0-10_2.11:2.1.0,com.databricks:spark-avro_2.11:3.2.0 pyspark-shell'

In [2]:
from pytwitter import StreamApi
import json
import os
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 


In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pykafka import KafkaClient

In [14]:
client = KafkaClient()

In [23]:
producer = client.topics["test1"].get_producer()

In [28]:
class MyStream(StreamApi):
    def __init__(self, bearer_token):
        try:
            StreamApi.__init__(self, bearer_token=bearer_token)
            self.__file_index = 0
        except KeyError as e:
            print("error on_tweet: %s" % str(e))

    def on_tweet(self, tweet):
        print(tweet)
        tweets_file_path = 'tweets_json/{}.json'.format(self.__file_index)
        with open(tweets_file_path, 'w', encoding='utf-8') as outfile:
                json.dump(tweet, outfile, ensure_ascii=False)
        while os.path.exists(tweets_file_path) and os.stat(tweets_file_path).st_size > 2**10:
            self.__file_index += 1 
            tweets_file_path = 'tweets_json/{}.json'.format(self.__file_index)

        try:
            text = tweet["data"]["text"]
            print(text + "\n")
            producer.produce(bytes(json.dumps(text), "utf-8"))
        except KeyError as e:
            print("error on_tweet: %s" % str(e))

            

In [5]:
rules = {
        "add": [
            {"value": "cat has:media", "tag": "cats with media"},
            {"value": "cat has:media -grumpy", "tag": "happy cats with media"}
        ]
     }

In [29]:
stream = MyStream(bearer_token=credentials.BEARER_TOKEN)
stream.manage_rules(rules=rules)
stream.search_stream(return_json=True)
# stream.sample_stream(return_json=True)

{'data': {'id': '1568976938970259457', 'text': '#PS4share\n\nhttps://t.co/CGPcHLlQSW https://t.co/180B918BRF'}, 'matching_rules': [{'id': '1545037781386153986', 'tag': 'happy cats with media'}, {'id': '1545037781386153987', 'tag': 'cats with media'}]}
#PS4share

https://t.co/CGPcHLlQSW https://t.co/180B918BRF

{'data': {'id': '1568976942493503489', 'text': 'RT @megazaru: 散歩中に黒猫を見たのだけど 肉眼では見えてるのに 何回撮っても全然写らなくて There is no cat in this image @Thereisnocat_ になっちゃった https://t.co/yGJgokM2Ma'}, 'matching_rules': [{'id': '1545037781386153986', 'tag': 'happy cats with media'}, {'id': '1545037781386153987', 'tag': 'cats with media'}]}
RT @megazaru: 散歩中に黒猫を見たのだけど 肉眼では見えてるのに 何回撮っても全然写らなくて There is no cat in this image @Thereisnocat_ になっちゃった https://t.co/yGJgokM2Ma

{'data': {'id': '1568976939582627840', 'text': "RT @captainspacefox: Sad cat dance, but that's not a cat! https://t.co/DV8l6co1cU"}, 'matching_rules': [{'id': '1545037781386153986', 'tag': 'happy cats with media'}, {'id': '1545037781386

KeyboardInterrupt: 